<div style="font-family: 'Courier New', Courier, monospace; font-size: 1.8em; color: #f6f8fa; background-color: #24292e; border: 2px solid #444d56; border-radius: 6px; padding: 10px 20px; margin-top: 20px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
    <span style="font-weight: 600; font-size: 1.1em; letter-spacing: 0.5px; color: #f0f6fc;">2. Importing the necessary libraries</span>
</div>


In [9]:
%%capture
%load_ext autoreload
%autoreload 2

from data_generator import DataGenerator as DataGenerator
from matplotlib import pyplot as plt
import pandas as pd
import ipywidgets as widgets
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'
from IPython.display import display

from vnf_model import *
from slice_model import *
from resource_allocation import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

In [10]:
ran_data_gen = DataGenerator('./net_model_dataset/ran/input_dataset.pkl', './net_model_dataset/ran/output_dataset.pkl', vnf_type='RAN', norm_type='minmax')
ran_model = VNF_Model('ran', 6, [64, 32, 16], 5)
ran_model.to(device)
ran_model.load_weights('./data/saved_weights/ran/model.pth')

ovs_data_gen = DataGenerator('./net_model_dataset/ovs/input_dataset.pkl', './net_model_dataset/ovs/output_dataset.pkl', vnf_type='OvS', norm_type='minmax')
ovs_model = VNF_Model('ovs', 6, [64, 32, 16], 5)
ovs_model.to(device)
ovs_model.load_weights('./data/saved_weights/ovs/model.pth')

upf_data_gen = DataGenerator('./net_model_dataset/upf/input_dataset.pkl', './net_model_dataset/upf/output_dataset.pkl', vnf_type='UPF', norm_type='minmax')
upf_model = VNF_Model('upf', 6, [32, 16], 5)
upf_model.to(device)
upf_model.load_weights('./data/saved_weights/upf/model.pth')

data_gens = [upf_data_gen, ovs_data_gen, ran_data_gen]
vnf_models = [upf_model, ovs_model, ran_model]
slice_model = SliceModel(vnf_models, data_gens)


In [ ]:
resource_allocation = {'UPF': 200, #CPU (millicores)
                       'OVS': 50, #Throughput (Mbps)
                       'RAN': 1000} #CPU (millicores)
input_throughput = 35

output_throughput = slice_model.predict_throughput(res = list(resource_allocation.values()),
                                                    input_throughput= input_throughput,
                                                    differentiable=0,
                                                    res_normalized=False)
print(output_throughput)

In [32]:
input_throughput = 30
output_throughput = 10
resource_allocation, _, _, _, _ = MicroOpt(slice_model, input_throughput, output_throughput, verbose=0)
print(resource_allocation)

Init weights: [1.   0.5  0.35]
[[0.99999905 0.49999952 0.34999964]]


In [33]:
output_throughput = slice_model.predict_throughput(list(resource_allocation[0]), input_throughput, res_normalized=True)
print(output_throughput)


20.1007137298584
